In [ ]:
!pip install -q pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 16.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import kagglehub
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import pymorphy3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# **Подготовка датасета**

In [ ]:
!gdown 14Kz1V1bOsfg0nGGujimcI3UdbrbFJccF
!gdown 153ncZ9LM7Ty2XyNUPU9Pg4oUxZITQtk0
!gdown 10SSwrsnIz1fWl6Nm-tq5upf5WMl5i7FH

Downloading...
From: https://drive.google.com/uc?id=14Kz1V1bOsfg0nGGujimcI3UdbrbFJccF
To: /content/twitter_data_1.csv
100% 1.77M/1.77M [00:00<00:00, 24.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=153ncZ9LM7Ty2XyNUPU9Pg4oUxZITQtk0
To: /content/twitter_data_2.csv
100% 1.26M/1.26M [00:00<00:00, 25.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=10SSwrsnIz1fWl6Nm-tq5upf5WMl5i7FH
To: /content/twitter_data_3.csv
100% 3.19M/3.19M [00:00<00:00, 84.8MB/s]


In [ ]:
path_twitter = kagglehub.dataset_download("thorinhood/russian-twitter-sentiment")
path_1 = path_twitter + "/negative.csv"
path_2 = path_twitter + "/positive.csv"
path_3 = kagglehub.dataset_download("blackmoon/russian-language-toxic-comments") + "/labeled.csv"

100%|██████████| 18.9M/18.9M [00:00<00:00, 145MB/s]

Extracting files...


100%|██████████| 1.49M/1.49M [00:00<00:00, 88.6MB/s]

Extracting files...


## **Читаем файлы в переменные**

In [ ]:
twitter_data_1 = pd.read_csv("/content/twitter_data_1.csv")

In [ ]:
twitter_data_2 = pd.read_csv("/content/twitter_data_2.csv")

In [ ]:
twitter_data_3 = pd.read_csv("/content/twitter_data_3.csv")

In [ ]:
twitter_data_negative = pd.read_csv(path_1, sep=";", names=['id', 'nick', 'text', 'label', 'a', 'b', 'c', 'd', 'f', 'g', 'h'])

In [ ]:
twitter_data_positive = pd.read_csv(path_2, sep=";", names=['id', 'nick', 'text', 'label', 'a', 'b', 'c', 'd', 'f', 'g', 'h'])

In [ ]:
twitter_data_toxic = pd.read_csv(path_3, sep=",")

## **Обрабатываем данные**

In [ ]:
twitter_data_1.drop(columns=["Unnamed: 0", "id"], inplace=True)

In [ ]:
twitter_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13392 entries, 0 to 13391
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    13392 non-null  object
 1   label   13392 non-null  object
dtypes: object(2)
memory usage: 209.4+ KB


In [ ]:
twitter_data_3['label'].unique()

array(['negative', 'speech', 'skip', 'neutral', 'positive'], dtype=object)

In [ ]:
twitter_data_1[twitter_data_1['label'] == 'skip']

,text,label
0,@varlamov @McFaul На,skip
17,@shn_shah Кеча топилмагунча уйқим кемаганди😅\r...,skip
22,@viktoshamurrr7 @CMEPTU___HET @derella9 Кладби...,skip
28,это как то странно чтлио,skip
42,@avantroux @Plesk7 @mix87074735444 Услышал,skip
...,...,...
13338,@Bazuulaa Зда чинь замын зорчих хэсгээс гараад...,skip
13344,#Алмазная #резка Москва и область https://t.co...,skip
13348,"@AlexGenM Кошкины,бее,но она мою тарелку может...",skip
13376,К нам пришли гости на огонёк https://t.co/DWKF...,skip


In [ ]:
twitter_data_2

,label,text
0,neutral,Прорвём информационную блокаду изнутри.
1,negative,"Никогда у меня не будет ""одного приложения для..."
2,skip,"Кури-и тебя не укусит злая собака, потому что ..."
3,neutral,"Есть 3 типа людей:\r\nУмные, которые делают вс..."
4,neutral,мегафон чет накрыло
...,...,...
6945,positive,"мы знаем , что мы лучшие(с):))"
6946,neutral,Раз такая ежовая тема☺️😊❤️
6947,neutral,так вот оно что...)
6948,positive,Бляяя...)) Вот это был номер...))) Такого я от...


In [ ]:
twitter_data_3

,label,text
0,negative,"А попа подозревала давно,что ты с кавказа..пер..."
1,speech,З прошедшим Днем Ангела))))))))
2,skip,Два дня до отлёта с острова!!!!!!!
3,negative,"Блин, почему эта жизнь столь не справедлива (((("
4,skip,где еще встречать свой день рождения как не на...
...,...,...
21263,neutral,"Анастасия, у меня есть друг, с которым вы хоро..."
21264,neutral,Колька пошли гулять!!?
21265,neutral,Ура! Золото дают бесплатно!Напиши это в 4 комм...
21266,speech,"С Праздником, Ксюнь! \r\nЖенского счастья тебе..."


In [ ]:
twitter_data_negative

,id,nick,text,label,a,b,c,d,f,g,h
408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0
...,...,...,...,...,...,...,...,...,...,...,...
425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [ ]:
twitter_data_positive

,id,nick,text,label,a,b,c,d,f,g,h
408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...
411368729235054592,1386912922,diminlisenok,"Спала в родительском доме, на своей кровати......",1,0,0,0,1497,56,34,2
411368729424187392,1386912922,qilepocagotu,RT @jebesilofyt: Эх... Мы немного решили сокра...,1,0,1,0,692,225,210,0
411368796537257984,1386912938,DennyChooo,"Что происходит со мной, когда в эфире #proacti...",1,0,0,0,4905,448,193,13
411368797447417856,1386912938,bedowabymir,"""Любимая,я подарю тебе эту звезду..."" Имя како...",1,0,0,0,989,254,251,0


In [ ]:
twitter_data_toxic['toxic'] = twitter_data_toxic['toxic'] * -2 + 2

In [ ]:
twitter_data_toxic.rename(columns={"comment" : 'text', 'toxic' : 'label'}, inplace=True)

In [ ]:
twitter_data_negative.drop(columns=['id', 'nick', 'a', 'b', 'c', 'd', 'f', 'g', 'h'], inplace=True)
twitter_data_positive.drop(columns=['id', 'nick', 'a', 'b', 'c', 'd', 'f', 'g', 'h'], inplace=True)

In [ ]:
twitter_data_negative = twitter_data_negative.set_index(np.arange(twitter_data_negative.shape[0]))

In [ ]:
twitter_data_positive = twitter_data_positive.set_index(np.arange(twitter_data_positive.shape[0]))

In [ ]:
twitter_data_negative['label'] = twitter_data_negative['label'] + 1
twitter_data_positive['label'] = twitter_data_positive['label'] + 1

In [ ]:
sum_of_examples = twitter_data_1.shape[0] + twitter_data_2.shape[0] + twitter_data_3.shape[0] + twitter_data_negative.shape[0] + twitter_data_positive.shape[0] + twitter_data_toxic.shape[0]
sum_of_examples

282856

## **Собираем всё в один датасет**

In [ ]:
twitter_data = pd.concat([twitter_data_2, twitter_data_3.set_index(np.arange(twitter_data_2.shape[0], twitter_data_2.shape[0] + twitter_data_3.shape[0]))], axis=0)
twitter_data

,label,text
0,neutral,Прорвём информационную блокаду изнутри.
1,negative,"Никогда у меня не будет ""одного приложения для..."
2,skip,"Кури-и тебя не укусит злая собака, потому что ..."
3,neutral,"Есть 3 типа людей:\r\nУмные, которые делают вс..."
4,neutral,мегафон чет накрыло
...,...,...
28213,neutral,"Анастасия, у меня есть друг, с которым вы хоро..."
28214,neutral,Колька пошли гулять!!?
28215,neutral,Ура! Золото дают бесплатно!Напиши это в 4 комм...
28216,speech,"С Праздником, Ксюнь! \r\nЖенского счастья тебе..."


In [ ]:
twitter_data = pd.concat([twitter_data, twitter_data_1.set_index(np.arange(twitter_data.shape[0], twitter_data.shape[0] + twitter_data_1.shape[0]))], axis=0)
twitter_data

,label,text
0,neutral,Прорвём информационную блокаду изнутри.
1,negative,"Никогда у меня не будет ""одного приложения для..."
2,skip,"Кури-и тебя не укусит злая собака, потому что ..."
3,neutral,"Есть 3 типа людей:\r\nУмные, которые делают вс..."
4,neutral,мегафон чет накрыло
...,...,...
41605,negative,все пора спать пиздец словила шизу
41606,neutral,такими темпами я создам новую секту или органи...
41607,neutral,"Ты смотрела аниме, завернувшись в одеяло ,пока..."
41608,negative,@AlyonaPikachu Пиздануться


## Убираем ненужные таргеты

In [ ]:
twitter_data[twitter_data['label'] == 'skip']['text'].iloc[83]

'О нас думают плохо лишь те, кто хуже нас, а те кто лучше нас… Им просто не до нас…'

In [ ]:
twitter_data = twitter_data[twitter_data['label'] != 'skip']

In [ ]:
twitter_data = twitter_data[twitter_data['label'] != 'neutral']

In [ ]:
twitter_data[twitter_data['label'] == 'speech']['text'].iloc[394]

'Христос Воскрес!с праздничком,Юль!'

In [ ]:
twitter_data = twitter_data[twitter_data['label'] != 'speech']

In [ ]:
twitter_data['label'].unique()

array(['negative', 'positive'], dtype=object)

## Правим лейблы

In [ ]:
label_to_number = {'negative': 0, 'positive': 2}

func = np.vectorize(lambda x: label_to_number[x])

twitter_data['label'] = func(twitter_data['label'])
twitter_data

,label,text
1,0,"Никогда у меня не будет ""одного приложения для..."
8,0,Просто в дрожь бросает. Как можно быть такими ...
9,2,"косяк, я соскучилась по тебе ("
10,0,Плсторел :'(
11,0,бляять да забей ты ужеееееее!!!!!
...,...,...
41600,2,@_sasukedaisuki_ я не знаю что сказать просто\...
41602,0,ладно всё делаю это ваше сраное дз
41603,0,66-летняя женщина пострадала на пожаре на улиц...
41605,0,все пора спать пиздец словила шизу


In [ ]:
twitter_data['label'].unique()

array([0, 2])

In [ ]:
twitter_data['label'].value_counts()

,count
label,
2,8524
0,6952


## Формируем итоговый датасет

In [ ]:
data_size = 1

In [ ]:
# data = pd.concat([twitter_data.sample(round(twitter_data.shape[0] * data_size)),
#                   twitter_data_toxic.sample(round(twitter_data_toxic.shape[0] * data_size)),
#                   twitter_data_negative.sample(round(twitter_data_negative.shape[0] * data_size)),
#                   twitter_data_positive.sample(round(twitter_data_positive.shape[0] * data_size))])

data = pd.concat([twitter_data_toxic,
                  twitter_data_negative.sample(round(twitter_data_negative.shape[0] * data_size)),
                  twitter_data_positive.sample(round(twitter_data_positive.shape[0] * data_size))])

In [ ]:
data

,text,label
0,"Верблюдов-то за что? Дебилы, бл...\n",0.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",0.0
2,Собаке - собачья смерть\n,0.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",0.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",0.0
...,...,...
33,"я рааааааад,ведь сегодня впереди вся ночь,сери...",2.0
18020,"""Сегодня видел, как мужик елку выносил... Слаб...",2.0
63698,@iboRuliT @Urtin Еееее!! Я как-то нашёл дома н...,2.0
45818,"ахах,даже поход в музей может стать веселой пр...",2.0


In [ ]:
data['label'].value_counts()

,count
label,
1.0,124497
0.0,116749


In [ ]:
data['label'] = data['label'] / 2
data

,text,label
0,"Верблюдов-то за что? Дебилы, бл...\n",0.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",0.0
2,Собаке - собачья смерть\n,0.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",0.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",0.0
...,...,...
33,"я рааааааад,ведь сегодня впереди вся ночь,сери...",1.0
18020,"""Сегодня видел, как мужик елку выносил... Слаб...",1.0
63698,@iboRuliT @Urtin Еееее!! Я как-то нашёл дома н...,1.0
45818,"ахах,даже поход в музей может стать веселой пр...",1.0


In [ ]:
data.to_csv("/content/drive/MyDrive/prepared_data.csv")

# **Приводим данные к божескому виду**

## Чтение данных

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/prepared_data.csv")

In [ ]:
data.drop(columns=["Unnamed: 0"], inplace=True)

In [ ]:
data['label'] = data['label'].astype('int64')

In [ ]:
data

,text,label
0,"Верблюдов-то за что? Дебилы, бл...\n",0
1,"Хохлы, это отдушина затюканого россиянина, мол...",0
2,Собаке - собачья смерть\n,0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",0
...,...,...
241241,как же прекрасно быть неинтересным и посредств...,1
241242,Всем удачных выходных) Не грустите и радуйтесь...,1
241243,@chernyatkina я заебался у кабинета стоять))у ...,1
241244,@Your_Dictator @KingRalf_MOCKBA какой Стас сег...,1


## Подготовка текстовых данных

In [ ]:
def clean(text):
    text = text.replace("ё", "е").replace("RT ", "")
    # text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    # text = re.sub('@[^\s]+','USER', text)
    text = re.sub('[^а-яА-Я]+', ' ', text)
    text = re.sub(' +',' ', text)
    return text.strip()

clean_text = np.vectorize(clean)

In [ ]:
data['preproc_text'] = clean_text(data['text'])

In [ ]:
data

,text,label,preproc_text
0,"Верблюдов-то за что? Дебилы, бл...\n",0,Верблюдов то за что Дебилы бл
1,"Хохлы, это отдушина затюканого россиянина, мол...",0,Хохлы это отдушина затюканого россиянина мол в...
2,Собаке - собачья смерть\n,0,Собаке собачья смерть
3,"Страницу обнови, дебил. Это тоже не оскорблени...",0,Страницу обнови дебил Это тоже не оскорбление ...
4,"тебя не убедил 6-страничный пдф в том, что Скр...",0,тебя не убедил страничный пдф в том что Скрипа...
...,...,...,...
241241,как же прекрасно быть неинтересным и посредств...,1,как же прекрасно быть неинтересным и посредств...
241242,Всем удачных выходных) Не грустите и радуйтесь...,1,Всем удачных выходных Не грустите и радуйтесь ...
241243,@chernyatkina я заебался у кабинета стоять))у ...,1,я заебался у кабинета стоять у тебя мама у каб...
241244,@Your_Dictator @KingRalf_MOCKBA какой Стас сег...,1,какой Стас сегодня агрессивно тупой фэйспалма ...


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
# stop_words = set(stopwords.words('russian'))
stop_words = set(stopwords.words('russian')) - {'не', 'нет', 'ни'}
punctuation = set(string.punctuation)

# stop_words = set([word for word in stop_words if (word != 'не' and word != 'нет' and word != 'ни')])
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


{'а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так',
 'такой',
 'там',
 

In [ ]:
# def clean_stop_words(text):
#     words = word_tokenize(text.lower())
#     # words = word_tokenize(text)
#     filtered_words = [word for word in words if word not in stop_words and word not in punctuation]
#     return " ".join(filtered_words)

# data['clean_stop_words'] = data['preproc_text'].apply(clean_stop_words)
data['clean_stop_words'] = data['preproc_text']

In [ ]:
data

,text,label,preproc_text,clean_stop_words
0,"Верблюдов-то за что? Дебилы, бл...\n",0,Верблюдов то за что Дебилы бл,Верблюдов то за что Дебилы бл
1,"Хохлы, это отдушина затюканого россиянина, мол...",0,Хохлы это отдушина затюканого россиянина мол в...,Хохлы это отдушина затюканого россиянина мол в...
2,Собаке - собачья смерть\n,0,Собаке собачья смерть,Собаке собачья смерть
3,"Страницу обнови, дебил. Это тоже не оскорблени...",0,Страницу обнови дебил Это тоже не оскорбление ...,Страницу обнови дебил Это тоже не оскорбление ...
4,"тебя не убедил 6-страничный пдф в том, что Скр...",0,тебя не убедил страничный пдф в том что Скрипа...,тебя не убедил страничный пдф в том что Скрипа...
...,...,...,...,...
241241,как же прекрасно быть неинтересным и посредств...,1,как же прекрасно быть неинтересным и посредств...,как же прекрасно быть неинтересным и посредств...
241242,Всем удачных выходных) Не грустите и радуйтесь...,1,Всем удачных выходных Не грустите и радуйтесь ...,Всем удачных выходных Не грустите и радуйтесь ...
241243,@chernyatkina я заебался у кабинета стоять))у ...,1,я заебался у кабинета стоять у тебя мама у каб...,я заебался у кабинета стоять у тебя мама у каб...
241244,@Your_Dictator @KingRalf_MOCKBA какой Стас сег...,1,какой Стас сегодня агрессивно тупой фэйспалма ...,какой Стас сегодня агрессивно тупой фэйспалма ...


In [ ]:
morph = pymorphy3.MorphAnalyzer()

def lemmatizer(text, morph):
    pymorphy_results = list(map(lambda x: morph.parse(x), text.split(' ')))
    return ' '.join([res[0].normal_form for res in pymorphy_results])

lemmatizer_vec = np.vectorize(lemmatizer)

In [ ]:
data['lemmatized_text'] = lemmatizer_vec(data['clean_stop_words'], morph)

In [ ]:
def clear_void_rows(data):
    data_with_void = data[data['lemmatized_text'] == ""].index
    data = data.iloc[[id for id in data.index if id not in data_with_void]]
    return data

In [ ]:
finally_data = clear_void_rows(data)

In [ ]:
finally_data.to_csv("/content/drive/MyDrive/lemmatized_data_with_ne.csv")

## Векторизация текстов

In [ ]:
finally_data = pd.read_csv("/content/drive/MyDrive/lemmatized_data_with_ne.csv")

### Эксперименты с данными

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(finally_data['lemmatized_text'], finally_data['label'], test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=18000,
    min_df=3,
    max_df=0.8,
    sublinear_tf=True
)

X_train_vec = tfidf_vectorizer.fit_transform(X_train)
X_test_vec = tfidf_vectorizer.transform(X_test)

In [ ]:
# pca_transform = PCA(n_components=4000)
# X_train_vec = pca_transform.fit_transform(X_train_vec)
# X_test_vec = pca_transform.transform(X_test_vec)

In [ ]:
tfidf_vectorizer.get_feature_names_out()[:100]

array(['аа', 'ааа', 'ааа как', 'ааа ну', 'аааа', 'аааа как', 'ааааа',
       'аааааа', 'ааааааа', 'аааааааа', 'ааааааааа', 'аааааааааа',
       'ааааааааааа', 'аахи', 'аахха', 'абзац', 'абонент', 'абсолютно',
       'абсолютный', 'абу', 'ава', 'аварийный', 'авария', 'аватар',
       'аватарка', 'август', 'авто', 'автобус', 'автограф', 'автомат',
       'автомат по', 'автоматически', 'автоматический', 'автомобиль',
       'автор', 'авторитет', 'авторский', 'автошкола', 'ага', 'агент',
       'агентство', 'агрессивный', 'агрессия', 'ад', 'адам', 'адвокат',
       'адекватный', 'админ', 'администрация', 'адов', 'адовый', 'адрес',
       'адски', 'адский', 'аж', 'аж до', 'аж на', 'ажила', 'аз', 'азаза',
       'азазаза', 'азия', 'ай', 'ай ай', 'ай да', 'айпад', 'айпада',
       'айса', 'айтюнс', 'айфон', 'ак', 'академия', 'акк', 'аккаунт',
       'аккумулятор', 'аккуратно', 'аккуратный', 'акт', 'активно',
       'активность', 'активный', 'актриса', 'актуальный', 'актёр',
       'актёрский'

# Учим модели и смотрим качество

In [ ]:
!pip install -q catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00


In [ ]:
!pip install joblib

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from joblib import dump, load

ModuleNotFoundError: No module named 'catboost'

In [ ]:
log_model = LogisticRegression(
    solver='lbfgs',
    max_iter=100,
    n_jobs=-1
)

log_model.fit(X_train_vec, y_train)

y_pred = log_model.predict(X_test_vec)

print(classification_report(y_test, y_pred))
print(f"\nroc_auc_score\t{roc_auc_score(y_test, y_pred)}")

              precision    recall  f1-score   support

           0       0.74      0.71      0.73     23200
           1       0.74      0.77      0.76     25047

    accuracy                           0.74     48247
   macro avg       0.74      0.74      0.74     48247
weighted avg       0.74      0.74      0.74     48247


roc_auc_score	0.74063074265209


In [ ]:
tree_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=300,
    n_jobs=-1,
    verbose=10
)

tree_model.fit(X_train_vec, y_train)

y_pred = tree_model.predict(X_test_vec)

print(classification_report(y_test, y_pred))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 500building tree 2 of 500

building tree 3 of 500
building tree 4 of 500


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   16.9s


building tree 5 of 500
building tree 6 of 500


[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   31.6s


...

[Parallel(n_jobs=-1)]: Done 477 tasks      | elapsed: 56.4min


building tree 479 of 500
building tree 480 of 500
building tree 481 of 500
building tree 482 of 500
building tree 483 of 500
building tree 484 of 500
building tree 485 of 500
building tree 486 of 500
building tree 487 of 500
building tree 488 of 500
building tree 489 of 500
building tree 490 of 500
building tree 491 of 500
building tree 492 of 500
building tree 493 of 500
building tree 494 of 500
building tree 495 of 500
building tree 496 of 500
building tree 497 of 500
building tree 498 of 500
building tree 499 of 500
building tree 500 of 500


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 59.0min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.9s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    1.2s
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    1.6s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.0s
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed:    2.4s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    2.9s
[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed:    3.4s
[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed:    4.0s
[Parallel(n_jobs=2)]: Done 109 tasks      | elapsed:    4.6s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:    

              precision    recall  f1-score   support

           0       0.74      0.67      0.70     23200
           1       0.72      0.79      0.75     25047

    accuracy                           0.73     48247
   macro avg       0.73      0.73      0.73     48247
weighted avg       0.73      0.73      0.73     48247



[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:   22.5s finished


In [ ]:
cat_model = CatBoostClassifier(
    iterations=50,
    learning_rate=0.1,
    depth=6,
    l2_leaf_reg=3,
    loss_function='Logloss',
    eval_metric='AUC',
    class_weights=[1, 3],
    thread_count=-1,
    verbose=100,
)

cat_model.fit(X_train_vec, y_train)

y_pred = cat_model.predict(X_test_vec)

print(classification_report(y_test, y_pred))

0:	total: 560ms	remaining: 27.4s
49:	total: 26.7s	remaining: 0us
              precision    recall  f1-score   support

           0       0.89      0.04      0.09     23200
           1       0.53      1.00      0.69     25047

    accuracy                           0.54     48247
   macro avg       0.71      0.52      0.39     48247
weighted avg       0.70      0.54      0.40     48247



In [ ]:
score = {}

for treshold in np.linspace(0, 1, 10):
    predict = tree_model.predict_proba(X_test_vec)
    y_pred = [1 if pred[1] > treshold else 0 for pred in predict]

    score[treshold] = roc_auc_score(y_test, y_pred)

score = pd.DataFrame({"treshold": score.keys(),
                      "score": score.values()}).sort_values(by="score", ascending=False)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    1.5s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    1.9s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    2.4s
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    3.3s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    3.8s
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed:    4.2s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    4.7s
[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed:    5.2s
[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed:    5.8s
[Parallel(n_jobs=2)]: Done 109 tasks      | elapsed:    6.4s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:    7.0s
[Parallel(n_jobs=2)]: Done 141 tasks      | elapsed:    7.7s
[Para

In [ ]:
score

,treshold,score
5,0.555556,0.728092
4,0.444444,0.718343
6,0.666667,0.674869
3,0.333333,0.674352
2,0.222222,0.613325
7,0.777778,0.598777
1,0.111111,0.542802
8,0.888889,0.530464
0,0.000000,0.501185
9,1.000000,0.500000


In [ ]:
data = pd.DataFrame({
    "text" : ["Перестань снимать ролики", "Так держать, продолжай в том же духе", "умри", "А вот это было сказано по факту"],
    "label" : [0, 1, 0, 1]
})

In [ ]:
data

,text,label,lemmatized_text,predicted_pos,predicted_neg
0,Перестань снимать ролики,0,перестать снимать ролик,0.335551,0.664449
1,Так держать продолжай в том же духе,1,так держать продолжать в тот же дух,0.776543,0.223457
2,умри,0,умереть,0.013446,0.986554
3,А вот это было сказано по факту,1,а вот это быть сказать по факт,0.688394,0.311606


In [ ]:
def clean(text):
    text = text.replace("ё", "е").replace("RT ", "")
    # text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    # text = re.sub('@[^\s]+','USER', text)
    text = re.sub('[^а-яА-Я]+', ' ', text)
    text = re.sub(' +',' ', text)
    return text.strip()

clean_text = np.vectorize(clean)

In [ ]:
morph = pymorphy3.MorphAnalyzer()

def lemmatizer(text, morph):
    pymorphy_results = list(map(lambda x: morph.parse(x), text.split(' ')))
    return ' '.join([res[0].normal_form for res in pymorphy_results])

lemmatizer_vec = np.vectorize(lemmatizer)

In [ ]:
def clear_void_rows(data):
    data_with_void = data[data['lemmatized_text'] == ""].index
    data = data.iloc[[id for id in data.index if id not in data_with_void]]
    return data

In [ ]:
def prepare_data_for_model(data):
    preproced_data = clean_text(data)
    # data['text'] = data['text'].apply(clean_stop_words)
    preproced_data = lemmatizer_vec(preproced_data, morph)
    preproced_data = tfidf_vectorizer.transform(preproced_data)

    return preproced_data

In [ ]:
y_pred = log_model.predict_proba(data_for_predict)

In [ ]:
data['predicted_pos'] = y_pred.T[1]
data['predicted_neg'] = y_pred.T[0]

In [ ]:
comments = [
    "Это было просто потрясающе!",
    "Ужасный сервис, никогда больше не вернусь сюда.",
    "Довольно неплохо, но могло бы быть и лучше."
]

In [ ]:
data = pd.DataFrame({
    'text' : comments,
    'label' : [1, 0, 1]
})

In [ ]:
data['text'] = clean_text(data['text'])
# data['text'] = data['text'].apply(clean_stop_words)
data['lemmatized_text'] = lemmatizer_vec(data['text'], morph)
data = clear_void_rows(data)
data_for_predict = tfidf_vectorizer.transform(data['lemmatized_text'])

In [ ]:
y_pred = log_model.predict_proba(data_for_predict)

In [ ]:
data['predicted_pos'] = y_pred.T[1]
data['predicted_neg'] = y_pred.T[0]

In [ ]:
data

,text,label,lemmatized_text,predicted_pos,predicted_neg
0,Это было просто потрясающе,1,это быть просто потрясающе,0.795136,0.204864
1,Ужасный сервис никогда больше не вернусь сюда,0,ужасный сервис никогда большой не вернуться сюда,0.079376,0.920624
2,Довольно неплохо но могло бы быть и лучше,1,довольно неплохо но мочь бы быть и хороший,0.798066,0.201934


In [ ]:
twitter_data['label'] = twitter_data['label'] / 2

In [ ]:
twitter_data

,label,text
1,0.0,"Никогда у меня не будет ""одного приложения для..."
8,0.0,Просто в дрожь бросает. Как можно быть такими ...
9,1.0,"косяк, я соскучилась по тебе ("
10,0.0,Плсторел :'(
11,0.0,бляять да забей ты ужеееееее!!!!!
...,...,...
41600,1.0,@_sasukedaisuki_ я не знаю что сказать просто\...
41602,0.0,ладно всё делаю это ваше сраное дз
41603,0.0,66-летняя женщина пострадала на пожаре на улиц...
41605,0.0,все пора спать пиздец словила шизу


In [ ]:
X = prepare_data_for_model(twitter_data['text'])

In [ ]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 159890 stored elements and shape (15476, 18000)>

In [ ]:
data = tree_model.predict_proba(X).T

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.7s
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed:    0.8s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    1.0s
[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed:    1.2s
[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed:    1.6s
[Parallel(n_jobs=2)]: Done 109 tasks      | elapsed:    1.9s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:    2.3s
[Parallel(n_jobs=2)]: Done 141 tasks      | elapsed:    2.7s
[Para

In [ ]:
data

array([[0.49715493, 0.49617039, 0.61301784, ..., 0.41541826, 0.76547876,
        0.41632656],
       [0.50284507, 0.50382961, 0.38698216, ..., 0.58458174, 0.23452124,
        0.58367344]])

In [ ]:
twitter_data['predicted_label'] = [1 if x > 0.555556 else 0 for x in data[1]]

In [ ]:
roc_auc_score(twitter_data['label'], twitter_data['predicted_label'])

np.float64(0.6902178051115674)

In [ ]:
dump(log_model, "/content/drive/MyDrive/Colab Notebooks/saved_models/log_model.joblib")

['/content/drive/MyDrive/Colab Notebooks/saved_models/log_model.joblib']

In [ ]:
dump(tfidf_vectorizer, "/content/drive/MyDrive/Colab Notebooks/saved_models/tfidf_vectorizer.joblib")

['/content/drive/MyDrive/Colab Notebooks/saved_models/tfidf_vectorizer.joblib']